In [1]:
import sqlalchemy
import numpy as np
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt 
from flask import Flask, jsonify

In [2]:
engine = create_engine('sqlite:///Resources/hawaii.sqlite')

In [3]:
Base = automap_base()

In [4]:
Base.prepare(engine,reflect= True)

In [5]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [6]:
session = Session(engine)

In [7]:
app = Flask(__name__)

In [8]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f'Available Routes: <br/>'
        f'/api/v1.0/percipitation/<br/>'
        f'/api/v1.0/stations<br/>'
        f'/api/v1.0/tobs<br/>'
        f'/api/v1.0/2017-06-03<br/>'
        f'/api/v1.0/2017-06-03/2018-06-03'
        )

In [9]:
@app.route('/api/v1.0/precipitation')
def percipitation():
    # Query for the dates and precipitation observations
   # from the last year.
   # Retrieving the data from 12 months before 8/23/2017
   # (assuming current date is 8/23/2017)
    rain = session.query(Measurement.date, Measurement.prcp). \
    filter(Measurement.date>='2016-08-23').all()
    #dictionary using date as the key and prcp as the value
    prec_dict = {date: prcp for date, prcp in rain}
    return jsonify(prec_dict)

In [10]:
@app.route('/api/v1.0/stations')
def station():
     # Retrieve the names of all the active stations from Station
   # and count of observations
    active_stations = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).all()
    stations_list = list(np.ravel(active_stations))
    return jsonify(stations_list)

In [12]:
@app.route('/api/v1.0/tobs')
def tobs():
    # Return a json list of Temperature Observations (tobs)
   # for the previous year
   # Retrieving the data from 12 months before 8/23/2017
   # (assuming current date is 8/23/2017)
    tobs_data = session.query(Measurement.date,Measurement.tobs).filter(Measurement.date >= '2016-08-23').filter(Measurement.station=='USC00519281').all()
    tobs_dict = {dict:tobs for date, tobs in tobs_data}
    return jsonify(tobs_dict)

In [18]:
@app.route('/api/v1.0/<start_date>')
@app.route("/api/v1.0/<start_date>/<end_date>")
def calc_temps(start_date=None, end_date=None):
    if not end_date:
        results = session.query(func.min(Measurement.tobs),func.avg(Measurement.tobs),func.max(Measurement.tobs)).filter(Measurement.date >=start_date).all()
        return jsonify(list(np.ravel(results)))
    else:
        results = session.query(func.min(Measurement.tobs), func.avg(Measuremen.tobs), func.max(Measurement.tobs)).filter(Measurement.date>=start_date).filter(Measurement.date<=end_date).all()
        return jsonify(list(np.ravel(results)))
    

In [ ]:
#trip_temp_min, trip_temp_avg, trip_temp_max = calc_temps(str(input(“Enter a Start Date (yyyy-mm-dd): “)), \
                                                        #str(input(“Enter an End Date (yyyy-mm-dd): “)))[0]
#print(trip_temp_min, trip_temp_avg, trip_temp_max)
#print(“Minimum Temperature: {}“.format(trip_temp_min))
#print(“Average Temperature: {}“.format(trip_temp_avg))
#print(“Maximum Temperature: {}“.format(trip_temp_max))
if __name__ = '__main__':
    app.run(debug=True)